# Migrate Catalog Object Permissions
This notebook reads all object permissions from a source catalog and generates SQL commands
to grant the same privileges on the objects in a destination catalog.


In [ ]:

%md
## Configure Source and Destination Catalogs


In [ ]:

try:
    dbutils.widgets.text("1.source_catalog", "source_catalog")
    dbutils.widgets.text("2.destination_catalog", "destination_catalog")
    source_catalog = dbutils.widgets.get("1.source_catalog")
    destination_catalog = dbutils.widgets.get("2.destination_catalog")
except NameError:
    # When running as a standard Python script (e.g., for testing), define values here
    source_catalog = "source_catalog"
    destination_catalog = "destination_catalog"


In [ ]:

%md
## Read Schema and Volume Permissions From the Source Catalog


In [ ]:

schema_query = f"""
SELECT
  schema_name,
  grantee,
  privilege_type
FROM system.information_schema.schema_privileges
WHERE catalog_name = '{source_catalog}'
"""

volume_query = f"""
SELECT
  volume_schema,
  volume_name,
  grantee,
  privilege_type
FROM system.information_schema.volume_privileges
WHERE volume_catalog = '{source_catalog}'
"""

schema_df = spark.sql(schema_query)
volume_df = spark.sql(volume_query)

privileges = [
    {
        "schema": row["schema_name"],
        "type": "SCHEMA",
        "principal": row["grantee"],
        "privilege": row["privilege_type"],
    }
    for row in schema_df.collect()
] + [
    {
        "schema": row["volume_schema"],
        "name": row["volume_name"],
        "type": "VOLUME",
        "principal": row["grantee"],
        "privilege": row["privilege_type"],
    }
    for row in volume_df.collect()
]


In [ ]:

%md
## Generate GRANT Commands


In [ ]:

grant_cmds = []
volume_grant_cmds = []

for p in privileges:
    if p["type"] == "VOLUME":
        object_identifier = f"`{destination_catalog}`.`{p['schema']}`.`{p['name']}`"
        volume_grant_cmds.append(
            f"GRANT {p['privilege']} ON VOLUME {object_identifier} TO `{p['principal']}`;"
        )
    else:
        if p["schema"] == "information_schema":
            continue
        object_identifier = f"`{destination_catalog}`.`{p['schema']}`"
        grant_cmds.append(
            f"GRANT {p['privilege']} ON SCHEMA {object_identifier} TO `{p['principal']}`;"
        )


In [ ]:

%md
## Display GRANT Commands (Non-Volume)


In [ ]:

for cmd in grant_cmds:
    print(cmd)


In [ ]:

%md
## Display GRANT Commands for Volumes


In [ ]:

for cmd in volume_grant_cmds:
    print(cmd)